In [1]:
import numpy as np
import random
from tabulate import tabulate

In [4]:
def generate_grid_world(length, width,path_lenght,holes_number,Random_State):
    
    random.seed(Random_State)
    #store all cells in a list
    Grid_Cells = []
    for row in range(length):
        for col in range(width):
            Grid_Cells.append([row,col])


    #specify the number of holes in the gridworld
    
    #specify the start point as a random cell
    start = [random.randint(0, length), random.randint(0, width)]

    #create a path from start point
    """instead of defining start and goal points,
      we define just a start point and a random path with a random lenght to
       another point and name it as goal point"""
    
    def random_path(Start, Path_Lenght,length, width):
        
        Path = []
        Path.append(Start)
        for i in range(Path_Lenght):
            
            #there are two moves that take us on a random cell named Goal [1,0], [0,1]
            
            move = random.choice([[1,0], [0,1]])
            
            #update the start cell/point by the above move
            Start = [x + y for x, y in zip(Start, move)]
            
            #if the movement take us out of our gridworld, we reverse the change in the start point
            if Start[0] < 0 or Start[1] < 0 or Start[0] > length-1 or Start[1] > width-1:

                Start = [x - y for x, y in zip(Start, move)]

            else:
                
                #create a path history
                Path.append(Start)

        Goal = Start

        return Goal,Path
    

    GoalPath = random_path(start, path_lenght,length, width)

    goal = GoalPath[0]
    path = GoalPath[1]

    #now we must eliminate the path cells from the Grid_Cells to choose hole cells from remaining cells

    FreeCells = [x for x in Grid_Cells if x not in path]

    Holes = random.sample(FreeCells, holes_number)

    #Also, we can visualize our gridworld in a simple way

    def mark_holes(holes):
        marked_data = [["Hole" if [row, col] in holes else [row, col] for col in range(width)] for row in range(length)]
        return marked_data
    
    marked_matrix = mark_holes(Holes)

    print(tabulate(marked_matrix, tablefmt="grid"))

    
    return length, width, start, goal, Holes, path,Grid_Cells

In [6]:
environment = generate_grid_world(5, 4,4,4,39)
environment

+--------+--------+--------+--------+
| Hole   | [0, 1] | [0, 2] | [0, 3] |
+--------+--------+--------+--------+
| [1, 0] | [1, 1] | [1, 2] | [1, 3] |
+--------+--------+--------+--------+
| Hole   | [2, 1] | [2, 2] | [2, 3] |
+--------+--------+--------+--------+
| Hole   | [3, 1] | Hole   | [3, 3] |
+--------+--------+--------+--------+
| [4, 0] | [4, 1] | [4, 2] | [4, 3] |
+--------+--------+--------+--------+


(5,
 4,
 [1, 2],
 [4, 3],
 [[2, 0], [3, 2], [3, 0], [0, 0]],
 [[1, 2], [1, 3], [2, 3], [3, 3], [4, 3]],
 [[0, 0],
  [0, 1],
  [0, 2],
  [0, 3],
  [1, 0],
  [1, 1],
  [1, 2],
  [1, 3],
  [2, 0],
  [2, 1],
  [2, 2],
  [2, 3],
  [3, 0],
  [3, 1],
  [3, 2],
  [3, 3],
  [4, 0],
  [4, 1],
  [4, 2],
  [4, 3]])

In [17]:
pdi = probability_distribution(environment[0]*environment[1],'stochastic')
pdi

{0: [0.38158976079279394,
  0.07578185681660621,
  0.26161743839425017,
  0.2810109439963497],
 1: [0.11954532797520077,
  0.3284074406060263,
  0.2603008388774112,
  0.2917463925413618],
 2: [0.36476627568513637,
  0.1294656347456722,
  0.4330919249064269,
  0.07267616466276457],
 3: [0.15043969540757351,
  0.23849769857925024,
  0.27586119463295145,
  0.3352014113802248],
 4: [0.06807769707778932,
  0.2518949826566975,
  0.34606369002380405,
  0.3339636302417091],
 5: [0.2346014451649269,
  0.2550107136921477,
  0.2942737593731373,
  0.21611408176978805],
 6: [0.28072101170798075,
  0.09198696898931331,
  0.3317083554651717,
  0.29558366383753426],
 7: [0.2586772288258927,
  0.0793268495282182,
  0.2586471191705574,
  0.4033488024753318],
 8: [0.35550867535157166,
  0.03720132642078019,
  0.28430771819938216,
  0.322982280028266],
 9: [0.21611853015102137,
  0.16164176168757594,
  0.4760260752096314,
  0.14621363295177126],
 10: [0.3106538330658406,
  0.3277646682255666,
  0.25500137

In [15]:
def probability_distribution(grid_size,randomness):
    random.seed(40)
    
    #by this function we generate probabilities which their sum is equal to 1
    def generate_probabilities(n):

        numbers = [random.random() for _ in range(n)]
        total_sum = sum(numbers)
        scaled_numbers = [num / total_sum for num in numbers]
        
        return scaled_numbers
    
    cells_prob = {}
    if randomness == 'stochastic':
        for cell in range(grid_size):
            
            #we set the number of probs to 4 due to 4 possible action for each cell (go to its neighbors)
            probs = generate_probabilities(4)

            cells_prob[cell] = probs
    elif randomness == 'equal probable':

        for cell in range(grid_size):

            cells_prob[cell] = [0.25,0.25,0.25,0.25]
    
    elif randomness == 'deterministic':
        for cell in range(grid_size):

            cells_prob[cell] = [0,0,0,1]#[0.15,.15,0.1,0.6]


    #Note that we consider the correspondence between probabilities and actions as below:
    #probs = [p1, p2, p3, p4] ---> [[1,0],[-1,0],[0,1],[0,-1]]

    return cells_prob

def neighbor_cells(cell):

    grid_cells = environment[6]
    Actions = [[1,0],[-1,0],[0,1],[0,-1]]

    Neighbors = []
    Actions_Neighbors = []
    for action in Actions:

        neighbor = [x + y for x, y in zip(cell, action)]
        #if neighbor not in environment[4]:
        Neighbors.append(neighbor)
        Actions_Neighbors.append(action)

    return Neighbors, Actions_Neighbors

#Note
"""As we want to use monte carlo method for estimating the state values
   it has been assumed that we have not any knowledge about the environment.
   Therefore, we should consider the transitions into the holes cells
   (against the case of policy iteration)"""

def arbitrary_policy(randomness):
    random.seed(randomness)
    
    policy = {}
    policy_action = {}
    for state in environment[6]:

        if state not in environment[4]:

            neighbors = neighbor_cells(state)[0]
            Actions_Neighbors = neighbor_cells(state)[1]

            allowed_positions = []

            for neighbor in neighbors:
                
                if neighbor in environment[6]:
                    
                    allowed_positions.append(neighbor)
        
            next_state = random.choice(allowed_positions)
            policy_action = random.choice(allowed_positions)
    
            policy['{}'.format(state)] = next_state

    return policy

def generate_trajectory(policy):

    start = environment[2]


    

In [11]:
policy_0 = arbitrary_policy(42)
policy_0

{'[0, 1]': [0, 0],
 '[0, 2]': [1, 2],
 '[0, 3]': [1, 3],
 '[1, 0]': [1, 1],
 '[1, 1]': [1, 2],
 '[1, 2]': [0, 2],
 '[1, 3]': [2, 3],
 '[2, 1]': [1, 1],
 '[2, 2]': [3, 2],
 '[2, 3]': [2, 2],
 '[3, 1]': [4, 1],
 '[3, 3]': [3, 2],
 '[4, 0]': [4, 1],
 '[4, 1]': [3, 1],
 '[4, 2]': [3, 2],
 '[4, 3]': [3, 3]}

## First-visit MC prediction, for estimationg $V \approx v_{\pi}$

In [ ]:
#Note that here we want to evaluate just a fixed policy
# and so we are not trying to optimize it 
def monte_carlo_prediction(num_trials, policy):

    v = np.zeros((environment[6],1))
    returns = {} #np.zeros((environment[6],1))
    
    #Loop for ever (for each episode)
    for trial in range(num_trials):
        
        #generate an episode
        policy = arbitrary_policy(trial)

        #total reward
        G = 0

        for 






In [12]:
a = {}
a.keys() = [1,2]
a

SyntaxError: cannot assign to function call here. Maybe you meant '==' instead of '='? (983677337.py, line 2)